<a href="https://colab.research.google.com/github/vkjdinesh/NLP/blob/main/Test_March23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Change  the run time to GPU

In [ ]:
!pip install Pillow==10.1.0

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install Flask==3.0.0
!pip install flask-ngrok==0.0.25
!pip install nltk==3.8.1
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

Here please include your ngrok authentication token.

In [ ]:
!mkdir /root/.ngrok2/
!echo -e "version: '2'\nregion: jp\nauthtoken: 2AxYhubJCLtqHpl4wB9pgrvl9Gl_64EcqFrwYmasY1zq86SMj" > ~/.ngrok2/ngrok.yml

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
#float value you can update it to 32 if there is any error popup
pipe = pipe.to("cuda")

import os
import matplotlib.pyplot as plt

In [ ]:
!pip install Flask-Cors==4.0.0


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import matplotlib.pyplot as plt
import io

In [ ]:
import os
import io
from PIL import Image

from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

import nltk

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

html = """
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <style>
      :root {
        --clr-green: #a3e635;
        --clr-dark-blue: #172554;
        --clr-white: #f8fafc;
      }

      * {
        padding: 0;
        margin: 0;
        font-family: "Franklin Gothic Medium", "Arial Narrow", Arial, sans-serif;
        border-radius: 0px;
      }

      body {
        background-color: var(--clr-dark-blue);
      }

      .section {
        height: 100%;
      }

      .prompt {
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        gap: 50px;
      }

      .btn {
        width: 200px;
        height: 50px;
        border: none;
      }

      .prompt__title {
        color: var(--clr-green);
        font-size: 100px;
      }

      .prompt__textfield {
        width: 600px;
        height: 70px;
        padding: 10px;
        text-align: center;
        background-color: var(--clr-green);
        border: none;
      }

      .story {
        display: flex;
        justify-content: center;
        align-items: center;
        gap: 70px;
        color: var(--clr-white);
        margin: auto 70px;
      }

      .story__story {
        width: 600px;
        display: flex;
        flex-direction: column;
        justify-content: start;
        gap: 20px;
      }

      .story__story-title {
        text-align: center;
        font-size: 60px;
      }

      .story__story-line {
        width: 100%;
        height: 5px;
        background-color: var(--clr-green);
      }

      .sentence {
        font-size: 20px;
      }

      .sentence:hover {
        background-color: var(--clr-green);
        cursor: pointer;
      }

      .story__image {
        width: 600px;
        display: flex;
        flex-direction: column;
        justify-content: start;
        align-items: center;
        gap: 20px;
      }

      .story__image-title {
        font-size: 40px;
      }

      .box {
        height: 50px;
        width: 100%;
        color: black;
        text-align: center;
        background-color: var(--clr-green);
      }

      .story__image-img {
        width: 100%;
      }

      .mcq {
        display: flex;
        flex-direction: column;
        align-items: center;
        gap: 20px;
      }

      .mcq__title {
        font-size: 40px;
        color: white;
      }

      .mcq__line {
        width: 50%;
        height: 5px;
        background-color: var(--clr-green);
      }

      .mcq__text {
        font-size: 20px;
        width: 50%;
        color: white;
      }
    </style>
    <title>AI Web App</title>
  </head>
  <body>
    <div class="section prompt">
      <h1 class="prompt__title">Enter the Text prompt.</h1>
      <form>
        <input
          class="prompt__textfield"
          type="text"
          name="prompt"
          placeholder="Two dogs playing together on a beach"
        />
      </form>
      <button class="btn" onclick="submit()">Enter</button>
    </div>

    <div class="section story">
      <div class="story__story">
        <h1 class="story__story-title">Short Story</h1>
        <div class="story__story-line"></div>
        <div class="story__story-text"></div>
      </div>
      <div class="story__image">
        <h1 class="story__image-title">Image</h1>
        <div>
          <div class="box">
            Max and Bella, two inseparable dogs, were playing joyfully on a
            sun-kissed beach.
          </div>
          <img
            class="story__image-img"
            src="https://png.pngtree.com/thumb_back/fw800/background/20220111/pngtree-fighting-dogs-on-the-beach-two-angry-sociability-photo-image_26163740.jpg"
            alt="generated picture"
          />
        </div>
        <button class="btn" onclick="showMcq()">MCQ</button>
      </div>
    </div>

    <div class="section mcq">
      <h1 class="mcq__title">MCQs</h1>
      <div class="mcq__line"></div>
      <div class="mcq__text"></div>
    </div>

    <script src="https://code.jquery.com/jquery-3.7.1.min.js" integrity="sha256-/JqT3SQfawRcv/BIHPThkBvs0OEvtFFmqPF/lYI/Cxo=" crossorigin="anonymous"></script>
    <script>
        function submit() {
            console.log("hit js")
            let prompt = document.querySelector(".prompt__textfield").value;
            console.log(prompt);
            $.ajax({
                url: '/handle_prompt',
                type: 'POST',
                data: {prompt: prompt},
                success: function(response) {
                    story = response.result[0];
                    alert(story);
                    let div = document.querySelector(".story__story-text");
                    div.textContent = "";
                    story.forEach((sentence, x) => {
                        let p = document.createElement("p");
                        let br = document.createElement("br");

                        p.textContent = sentence;
                        p.classList.add("sentence");
                        p.onclick = getImage;

                        div.appendChild(p);
                        div.appendChild(br);
                    })

                    let storySection = document.querySelector(".story");
                    storySection.scrollIntoView({ behavior: "smooth" });
                    console.log("done creating sentences");

                    mcq = response.result[1];
                    div = document.querySelector(".mcq__text");
                    mcq.forEach((mcqq, x) => {
                        let p = document.createElement("p");

                        p.textContent = mcqq;

                        div.appendChild(p);
                    })
                },
                error: function(error) {
                    console.log("error from ajax: " + JSON.stringify(error));
                }
            });
        }

        function getImage(event) {
            let ele = event.target
            console.log(ele.innerHTML)
            let img = document.querySelector(".story__image-img");
            img.src = "/handle_image/" + ele.innerHTML;

            let caption = document.querySelector(".box");
            caption.innerHTML = ele.innerHTML;

            console.log("changed image")
          }

        function showMcq() {
          let mcqSection = document.querySelector(".mcq");
          mcqSection.scrollIntoView({ behavior: "smooth" });
        }
    </script>
  </body>
</html>
"""

@app.route('/handle_prompt', methods=['POST'])
def handle_prompt():
    print('hit py')
    prompt = request.form.get('prompt')
    prompt_template = f'''
    USER: {prompt + "1.give short story with 200 words. 2. generate 5 MCQs with options and answer"}

    ASSISTANT:
    '''
    response = lcpp_llm(prompt=prompt_template, max_tokens=1500, temperature=0.5, top_p=0.95,
                        repeat_penalty=1.2, top_k=150,
                        echo=True)

    res = response["choices"][0]["text"]
    res = res.split(prompt_template)[1] # getting only the AI's response
    print(res)

    try:
        story, mcq = res.split("1.")
    except ValueError:
        story, mcq = res.split("1)")

    if story.count(":") == 2:
        story = story.split(":")[1].strip()
    elif story.count(":") == 3:
        story = story.split(":")[2].strip()
    elif story.count(":") == 5:
        story = story.split(":")[5].strip()

    mcq = "1." + mcq

    nltk.download('punkt')
    story = nltk.sent_tokenize(story)
    mcq = nltk.sent_tokenize(mcq)
    story = story[:-1] # exclude the last sentence

    print(story, mcq)
    return jsonify(result=[story, mcq])



@app.route('/handle_image/<prompt>', methods=['GET'])
def handle_image(prompt):
    try:
        print('hit py')

        # Generate image using the GPT-Neo pipeline
        image = pipe(prompt + " Japanese Anime").images[0]

        # Save the generated image temporarily
        image.save("output_image.jpeg")

        # Open the image using PIL and convert to BytesIO
        with open("output_image.jpeg", "rb") as image_file:
            image_io = io.BytesIO(image_file.read())

        # Remove the temporary image file
        os.remove("output_image.jpeg")

        # Return the processed image
        return send_file(image_io, mimetype='image/jpeg')

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({'error': str(e)})

@app.route("/")
def home():
    return html

if __name__ == '__main__':
    app.run()